In [5]:
%pip install -U sagemaker

/opt/conda/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
You should consider upgrading via the '/opt/conda/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [6]:
!pwd

/root/roberta_with_tags


In [7]:
import sagemaker
from sagemaker.huggingface import HuggingFace

sess = sagemaker.Session()
role = sagemaker.get_execution_role()

s3_bucket = sess.default_bucket()

In [8]:
metric_defs = [
    {
        "Name": "eval loss",
        "Regex": "'eval_loss': (.*?),",
    },
    {
        "Name": "eval acc",
        "Regex": "'eval_accuracy': (.*?),",
    },
    {
        "Name": "eval f1",
        "Regex": "'eval_f1': (.*?),",
    },
    {
        "Name": "loss",
        "Regex": "'loss': (.*?),",
    },
    {
        "Name": "epoch",
        "Regex": "'epoch': (.*?)}",
    },
]

In [9]:
# hyperparameters, which are passed into the training job
hyperparameters={'epochs': 250,
                 'train-batch-size': 32,
                 'eval-batch-size': 32,
                 'save_steps': 1000,
                 'model-dir':'/opt/ml/checkpoints',
                 'save_total_limit': 2,
                 'learning_rate': 3e-4,
                 }

def minutes(m):
    return m*60

def hours(h):
    return h*60*60

# create the Estimator
huggingface_estimator = HuggingFace(
        entry_point='train.py',
        source_dir='.',
        checkpoint_s3_uri=f's3://{sess.default_bucket()}/tweet_and_tag/checkpoints',
        use_spot_instances=True,
        max_wait=hours(12), # This should be equal to or greater than max_run in seconds'
        max_run=minutes(60),
        metric_definitions=metric_defs,
        instance_type='ml.g4dn.xlarge',
        instance_count=1,
        role=role,
        transformers_version='4.6',
        pytorch_version='1.7',
        py_version='py36',
        hyperparameters = hyperparameters
)

In [ ]:
huggingface_estimator.fit(
  {'train': f's3://{s3_bucket}/tweet_and_tag/train',
   'test': f's3://{s3_bucket}/tweet_and_tag/test'}
)

2021-08-22 20:11:08 Starting - Starting the training job...
2021-08-22 20:11:32 Starting - Launching requested ML instancesProfilerReport-1629663068: InProgress
......
2021-08-22 20:12:32 Starting - Preparing the instances for training........

2021-07-31 12:41:22 Starting - Starting the training job...
2021-07-31 12:41:46 Starting - Launching requested ML instancesProfilerReport-1627735279: InProgress
...
2021-07-31 12:42:16 Starting - Preparing the instances for training.........
2021-07-31 12:43:46 Downloading - Downloading input data......
2021-07-31 12:44:47 Training - Downloading the training image...........bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2021-07-31 12:46:36,393 sagemaker-training-toolkit INFO     Imported framework sagemaker_pytorch_container.training
2021-07-31 12:46:36,414 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2021-07-31 12:46:39,472 sagemaker_pytorch_container.training INFO     Invoking user training script.
2021-07-31 12:46:40,081 sagemaker-training-toolkit INFO     Invoking user script

Training Env:

{
    "additional_framework_parameters": {},
    "channel_input_dirs": {
        "tes

Job ended with status 'Stopped' rather than 'Completed'. This could mean the job timed out or stopped early for some other reason: Consider checking whether it completed as you expect.


Training seconds: 3729
Billable seconds: 1119
Managed Spot Training savings: 70.0%
